Cài đặt thư viện

In [ ]:
!pip install datasets transformers accelerate git-lfs

Đăng ký tài khoản huggingface và tạo token để đăng nhập phục vụ việc upload và download mô hình và dữ liệu

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

Kiểm tra việc cài đặt đã hoàn thành và phiên bản

In [ ]:
import transformers
transformers.__version__

'4.33.3'

# Fine-tuning mô hình task trả lời câu hỏi


Notebook sẽ hướng dẫn finetune một mô hình ngôn ngữ đã được huấn luyện trước (pretrained model) [🤗 Transformers](https://github.com/huggingface/transformers) cho bài toán trả lời câu hỏi bằng phương pháp trích xuất (extractive question answering)

![Ví dụ](https://github.com/huggingface/notebooks/blob/main/examples/images/question_answering.png?raw=1)

**Note:** Mô hình chỉ sử dụng thông tin trong ngữ cảnh để trả lời, không sinh ra câu trả lời.

## Loading the dataset

Sử dụng thư viện [🤗 Datasets](https://github.com/huggingface/datasets) với 2 hàm `load_dataset` and `load_metric` để tải dữ liệu và tính toán độ chính xác của mô hình.  

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
squad_v2 = False # V2 là bản cải tiến của V1 được thêm vào những câu hỏi không có câu trả lời
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

`datasets` là đối tượng dạng [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) gồm 2 phần ***train*** và ***validation***

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Các truy cập một phần tử

In [ ]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

# Giải thích tên các trường:
- id: chỉ số
- title: tiêu đề
- context: ngữ cảnh của câu hỏi
- question: câu hỏi
- answers:
  * answer_start: là một mảng chứa các vị trí bắt đầu của câu trả lời. (có thể rỗng)
  * text: là một mảng chứa các câu trả lời tương ứng với từng vị trí ở mảng answer_start. (có thể rỗng)



## Tiền xử lý dữ liệu huấn luyện

Trước khi đưa dữ liệu vào mô hình ta cần tiền xử lý chúng. Để làm được điều này ta sẽ sử dụng một 🤗 Transformers `Tokenizer` để tokenize dữ liệu đầu vào.

Ta sử dụng phương thức `AutoTokenizer.from_pretrained` để tạo ra một đối tượng `Tokenizer`.

**Note:** Tokenizer có thể cần giống với mô hình sử dụng nhưng trong đa số trường hợp nên sử dụng tokenizer đi kèm với mô hình để đạt hiệu quả tốt nhất.


Tải tokenizer

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
### YOUR CODE HERE
### 1. Load tokenizer bằng phương thức `from_pretrained` và truyền vào tên của model
tokenizer = ...
### END

Thử nghiệm tokenizer

In [ ]:
tokenizer("What is your name?", "My name is Sylvain.")
# hoặc tokenizer.encode("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Kiểm tra ngược

In [ ]:
tokenizer.decode([101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102])

'[CLS] what is your name? [SEP] my name is sylvain. [SEP]'

In [ ]:
max_length = 384 # Độ dài tối đa của input đầu vào
doc_stride = 128 # Đoạn trùng nhau giữa 2 phần nếu như cắt

Kiểm tra việc sử dụng `max_length` và `max_length`

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Khi không cắt

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

396

Khi cắt

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Thêm vào tham số
* `truncation="only_second"` để chỉ cắt phần sau (context)
* `return_overflowing_tokens=True` để giữ lại và xử lý cả những phần bị cắt

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Kiểm tra xem việc thêm có giữ lại được phần bị cắt không

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

Kiểm tra ngược lại

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at notr

Vì việc trả lời câu hỏi ta cần phải biết được vị trí của câu trả lời trong context nên ta sử dụng thêm `return_overflowing_tokens=True` để biết được vị trí của các token.

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

**Note:** `[CLS]` là ký tự đặc biệt nên sẽ có vị trí bắt đầu và kết thúc là (0, 0)

Kiểm tra lại

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


`sequence_ids()` sẽ giúp ta biết mỗi token thuộc về câu nào khi ta truyền nhiều câu vào `tokenizer`

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

**Note:** `none` là giá trị đặc biệt khi token đó không thuộc về câu nào (thường là các token đặc biệt)

Tìm câu trả lời từ dữ liệu được tokenize

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

### YOUR CODE HERE
# Tìm vị trí của token bắt đầu của ngữ cảnh

token_start_index = ...

# Tìm vị trí token kết thúc của ngữ cảnh
token_end_index = ...

# Kiểm tra xem câu trả lời có nằm trong ngữ cảnh hiện tại không (do ngữ cảnh dài có thể bị chia nhỏ)
offsets = tokenized_example["offset_mapping"][0]
start_position = ...
end_position = ...

### END

23 26


Kiểm tra lại

In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


Ta cần padding nếu như ngữ cảnh hoặc context ngắn

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

Trong trường hợp không có câu trả lời ta sẽ đặt vị trí bắt đầu và vị trí kết thúc của câu trả lời vào vị trí của token `[CLS]`

In [ ]:
def prepare_train_features(examples):

    # Loại bỏ các khoảng trắng ở đầu hoặc cuối
    examples["question"] = [q.strip() for q in examples["question"]]

    # Tokenize dữ liệu
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )


    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Tạo mảng để lưu vị trí bắt đầu và kết thúc của câu trả lời
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # Các câu hỏi không có câu trả lời sẽ được đặt vào vị trí của token [CLS]
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Để biết vị trí của context và câu hỏi
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Do việc cắt câu nên một câu có thể nằm ở nhiều phần khác nhau ta lấy sample_index để tìm lại vị trí
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        ### YOUR CODE HERE
        # Nếu không có câu trả lời đưa vào vị trí cls_index
        if len(answers["answer_start"]) == 0:
            ...
        else:
            # Vị trí bắt đầu và kết thúc của câu trả lời
            start_char = ...
            end_char = ...

            # Tìm vị trí token bắt đầu của context
            token_start_index = ...
            ...

            # Tìm vị trí token kết thúc của context
            token_end_index = ...
            ...

            # Kiểm trả xem câu trả lời có nằm trong đoạn không
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                # Nếu không thì đưa vào vị trí cls_index như không có câu trả lời
                ...
                ...
            else:
                # Tìm vị trí của đoạn chứa câu trả lời
                ...
          ### END
    return tokenized_examples

Hàm sẽ làm việc với 1 hoặc nhiều example đồng thời

In [ ]:
features = prepare_train_features(datasets['train'][:5])

Sử dụng phương thức `map` để tạo ra dữ liệu được tokenize và xóa hết đi các cột của dữ liệu cũ

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

## Fine-tuning model

Tạo ra một đối tượng `AutoModelForQuestionAnswering` bằng cách gọi phương thức `from_pretrained`

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Cảnh báo kia thông báo rằng mô hình pretrain không phục vụ sẵn cho bài toán. Nên mô hình sẽ thêm và khởi tạo thêm 1 lớp ở cuối cùng của mô hình để giải quyết bài toán.

[`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) là một lớp quan trọng giúp ta chọn các siêu tham số cho mô hình

In [ ]:
from transformers import TrainingArguments
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    # push_to_hub=True,
)

Ta cần một data collator để batch dữ liệu lại trong quá trình huấn luyện trong trường hợp này ta chỉ cần sử dụng mặc định

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

Tạo ra đối tượng `Trainer` để huấn luyện mô hình

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Huấn luyện mô hình bằng phương thức train

In [ ]:
# Thử train do thời gian có hạn
trainer.train()

Lệnh để lưu mô hình

In [ ]:
trainer.save_model("test-squad-trained")

Lệnh để đẩy mô hình lên tài khoản cá nhân

In [ ]:
# trainer.push_to_hub()

## Đánh giá

Mô hình không trả lại trực tiếp câu trả lời mà chỉ đưa ra xác xuất về vị trí bắt đầu và kết thúc của câu trả lời.

In [ ]:
import torch
torch.cuda.empty_cache()
model = AutoModelForQuestionAnswering.from_pretrained("csarron/bert-base-uncased-squad-v1").to("cuda")
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


odict_keys(['loss', 'start_logits', 'end_logits'])

Ta không cần quan tâm loss khi đánh giá chỉ cần giữ lại `'start_logits'` và `'end_logits'`

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([32, 384]), torch.Size([32, 384]))

Lấy ra kết quả tốt nhất

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 46,  57,  89,  43, 118,  44,  72,  42,  44,  41,  73,  41,  80,  45,
         156,  35,  40,  45,  80,  58,  77,  74,  42,  53,  41,  35,  42,  88,
          44,  44,  27, 133], device='cuda:0'),
 tensor([ 47,  58,  81,  44, 118, 109,  75,  11, 109,  42,  76,  42,  83,  45,
         159,  35,  83,  45,  83,  60,   0,  74,  43,  54,  42,  35,  43,  91,
          45,  45,  28, 133], device='cuda:0'))

Phương pháp chọn tốt nhất không khả thi????

Nên chọn cặp tốt nhất

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
### YOUR CODE HERE
### Mỗi phần từ là 1 từ điển gồm có
### - score: điểm của câu trả lời là start_logits[i] + end_logits[j]
### - text: hiện tại tạm để rỗng

### END

Viết hoàn chỉnh

In [ ]:
def prepare_validation_features(examples):

    # Tiền xử lý
    examples["question"] = [q.strip() for q in examples["question"]]

    # Tokenize
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # Để lưu lại vị trí feature thuộc vào example nào
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):

        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # Lưu lại id
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Đặt bằng None nếu như token không nằm trong context
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

Sử dụng `map` để chuyển tương tự như train

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Đưa ra dự đoán bằng phương thức predict

In [ ]:
raw_predictions = trainer.predict(validation_features)

`Trainer` giấu các cột không sử dụng bởi mô hình các chung ta cần để hậu xử lý nên ta cần đặt lại

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

Hậu xử lý

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]

context = datasets["validation"][0]["context"]

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []

### YOUR CODE HERE
for start_index in start_indexes:
    for end_index in end_indexes:
        # Loại bỏ những cặp nằm ngoài context
        ...
        # Loại bỏ những cặp không tồn tại và độ dài lớn hơn max_answer_length
        ...
        # Kiểm tra xem đoạn có thỏa mãn không
        # mảng valid_answer lưu các phần tử là từ điển gồm score và text
        ...
### END
valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 9.042664, 'text': 'Denver Broncos'},
 {'score': 7.7837133,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 6.7467585, 'text': 'Carolina Panthers'},
 {'score': 5.914267, 'text': 'Broncos'},
 {'score': 4.6553164,
  'text': 'Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 4.487817, 'text': 'Denver'},
 {'score': 3.6236343,
  'text': 'The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 3.2325382,
  'text': 'American Football Conference (AFC) champion Denver Broncos'},
 {'score': 2.6190052,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10'},
 {'score': 2.5684674,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina'},
 {'score': 2.4616265, 'text': 'AFC) champion Denver Broncos'},
 {'score': 2.3646836,
  'text': 'The American Football Conference (A

So sánh với câu trả lời đúng

In [ ]:
datasets["validation"][0]["answers"]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

Do là một dữ liệu có thể được chia làm nhiều feature nên ta cần phải tổng hợp lại và chọn ra câu trả lời tốt nhất.

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

Tạo ra hàm hậu xử lý

**Note:** cần sử dụng ngưỡng động (điểm của [CLS] để quyết định xem câu hỏi có câu trả lời hay không)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # Tạo từ điển để lưu đáp án
    predictions = collections.OrderedDict()

    # Log
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    ### YOUR CODE HERE
    for example_index, example in enumerate(tqdm(examples)):
        # Lấy tất cả feature ứng với dữ liệu
        feature_indices = features_per_example[example_index]

        min_null_score = None # Sử dụng là ngưỡng tối thiểu cho câu trả lời
        valid_answers = []

        context = example["context"]
        # Đi qua từng feature
        for feature_index in feature_indices:

            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            offset_mapping = features[feature_index]["offset_mapping"]

            # Cập nhật giá trị ngưỡng tối thiểu
            ...

            # Đi qua tất cả các cặp thỏa mãn và lưu vào valid_answers
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            ...

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer
    ### END
    return predictions

Áp dụng hàm vào dữ liệu được dự đoán ban đầu

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10784 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

Tải metric để đánh giá

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

<ipython-input-67-69495f532713>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 60.65279091769158, 'f1': 69.87707803246407}

In [ ]:
# trainer.push_to_hub()

Chia sẻ mô hình `"username/modelname"`:

```python
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("my-name/my-EQA-model")
```